<a href="https://colab.research.google.com/github/pavithrasenth/Hand_Detection/blob/main/Lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
df = pd.read_csv("medium_articles.csv")
texts = df['text'].dropna().tolist()

Saving medium_articles.csv to medium_articles (2).csv


In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

sentences = []
for t in texts[:5000]:
    for s in t.split("."):
        s = clean_text(s).strip()
        if len(s.split()):
            sentences.append(s)

In [8]:
all_words = [word for s in sentences for word in s.split()]
word_counts = Counter(all_words)
vocab = {w: i+2 for i, (w, _) in enumerate(word_counts.most_common())}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1

def encode_sentence(sentence):
    return [vocab.get(w, 1) for w in sentence.split()]

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class TextDataset(Dataset):
    def __init__(self, sentences, vocab, max_len=20):
        self.vocab = vocab
        self.max_len = max_len
        self.data = []
        for s in sentences:
            words = encode_sentence(s)
            if len(words) < 2:
                continue
            input_seq = words[:-1]
            target = words[-1]
            if len(input_seq) < max_len:
                input_seq = [0]*(max_len-len(input_seq)) + input_seq
            else:
                input_seq = input_seq[-max_len:]
            self.data.append((input_seq, target))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0]), torch.tensor(self.data[idx][1])

dataset = TextDataset(sentences, vocab)
train_data, test_data = train_test_split(dataset, test_size=0.1, random_state=42)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)


In [12]:
class LSTMModel(nn.Module):
    def _init_(self, vocab_size, embed_dim, hidden_dim):
        super(LSTMModel, self)._init_()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        output, (h, c) = self.lstm(x)
        out = self.fc(output[:, -1, :])
        return out

In [14]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


In [15]:
idx2word = {i: w for w, i in vocab.items()}

def predict_last_word(sentence):
    model.eval()
    words = encode_sentence(clean_text(sentence))
    if len(words) < 1:
        return None
    if len(words) < dataset.max_len:
        words = [0]*(dataset.max_len-len(words)) + words
    else:
        words = words[-dataset.max_len:]
    X = torch.tensor(words).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(X)
        predicted_idx = torch.argmax(output, dim=1).item()
    return idx2word.get(predicted_idx, "<UNK>")

In [18]:
def predict_last_word(sentence, top_k=5):
    model.eval()
    words = encode_sentence(clean_text(sentence))
    if len(words) < 1:
        return None

    input_seq = words[-max_len:]
    if len(input_seq) < max_len:
        input_seq = [0] * (max_len - len(input_seq)) + input_seq

    input_tensor = torch.tensor([input_seq]).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.softmax(output, dim=1)
        top_idx = torch.topk(probs, top_k).indices[0].tolist()

    inv_vocab = {idx: word for word, idx in vocab.items()}
    predictions = [inv_vocab.get(idx, "<UNK>") for idx in top_idx]

    return predictions


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(len(vocab), embed_dim=128, hidden_dim=256).to(device)


In [23]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/5, Loss: 7.8583
Epoch 2/5, Loss: 5.6452
Epoch 3/5, Loss: 3.7113
Epoch 4/5, Loss: 2.3076
Epoch 5/5, Loss: 1.3892


In [24]:
torch.save(model.state_dict(), "lstm_model.pth")
print("Model saved ")


Model saved 


In [25]:
model = LSTMModel(len(vocab), embed_dim=128, hidden_dim=256).to(device)
model.load_state_dict(torch.load("lstm_model.pth", map_location=device))
model.eval()
print("Model loaded")


Model loaded


In [27]:
max_len = 20


In [28]:
def predict_last_word(sentence, max_len=20, top_k=5):
    model.eval()
    words = encode_sentence(clean_text(sentence))
    if len(words) < 1:
        return None

    # Pad/truncate to match max_len
    input_seq = words[-max_len:]
    if len(input_seq) < max_len:
        input_seq = [0] * (max_len - len(input_seq)) + input_seq

    input_tensor = torch.tensor([input_seq]).to(device)

    with torch.no_grad():
        output = model(input_tensor)           # logits
        probs = torch.softmax(output, dim=1)   # probabilities
        top_idx = torch.topk(probs, top_k).indices[0].tolist()

    # Convert indices back to words
    inv_vocab = {idx: word for word, idx in vocab.items()}
    predictions = [inv_vocab.get(idx, "<UNK>") for idx in top_idx]

    return predictions


In [29]:
print("Prediction Example:")
print("Input: Deep learning is very")
print("Predicted last word:", predict_last_word("Deep learning is very"))


Prediction Example:
Input: Deep learning is very
Predicted last word: ['encouraging', 'agile', 'different', 'easy', 'important']
